In [9]:

import logging
from collections import namedtuple
import random
from copy import deepcopy
from itertools import accumulate
from operator import xor


Nimply = namedtuple("Nimply", "row, num_objects")

class Nim:
    def __init__(self, num_rows: int, k: int = None):
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self):
        return self._k
    
    #apply the chosen move by removing num_objects from the row
    def nimming(self, ply: Nimply):
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

Define the strategies that can be used during the game

In [10]:
"""Pick always the minimum(maximum) possible number of the lowest row"""
def dumb_PCI(state: Nim):
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], -m[1])))

"""remove from a random row a random number of objects"""
def pure_random(state: Nim):
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

"""optimal strategy"""
def nim_sum(state: Nim):
    *_, result = accumulate(state.rows, xor)
    return result

def cook_status(state: Nim):
    cooked = dict()
    cooked["possible_moves"] = [
        (r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k
    ]
    cooked["active_rows_number"] = sum(o > 0 for o in state.rows)
    cooked["shortest_row"] = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    cooked["longest_row"] = max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
    cooked["nim_sum"] = nim_sum(state)

    brute_force = list()
    for m in cooked["possible_moves"]:
        tmp = deepcopy(state)
        tmp.nimming(m)
        brute_force.append((m, nim_sum(tmp)))
    cooked["brute_force"] = brute_force

    return cooked

def optimal_startegy(state: Nim):
    data = cook_status(state)
    return next((bf for bf in data["brute_force"] if bf[1] == 0), random.choice(data["brute_force"]))[0]

In [11]:
NIM_SIZE = 6
NUM_MATCHES = 10

def eval_terminal(state: Nim):
    if not state:
        return -1
    else:
        return 0
    
def minmax(state: Nim,deep: int):
    val = eval_terminal(state)
    possible_moves = [
        (r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k
    ]
    
    if val != 0 or not possible_moves:
        return None, val
    evaluations = list()
    
    #deep pruning
    if NIM_SIZE >= 4 and deep>= 8:
        return None,1

    deep +=1

    for ply in possible_moves:
        
        tmp = deepcopy(state)
        tmp.nimming(ply)
        _, val = minmax(tmp, deep)
        evaluations.append((ply, -val))
        
        #alpha-beta pruning
        if -val == 1:
            break;
            

    return max(evaluations, key=lambda k: k[1])

In [12]:
"""Evaluation of minmax strategy"""
won = 0
nim = Nim(NIM_SIZE)
for i in range(NUM_MATCHES):
    deep = 0   
    player = 0
    while nim:
        if player == 0:
            ply, _ = minmax(nim,deep)
        else:
            ply = optimal_startegy(nim)

        nim.nimming(ply)
        player = 1 - player
        
    if player == 1:
        won += 1
        
    nim = Nim(NIM_SIZE)
        
win_rate = won / NUM_MATCHES
print(f"win rate = {win_rate}")

win rate = 0.0
